In [0]:
## TODO -- will need additional image preprocessing -- 2D coronal slices + corresponding masks 

## test using non onehot encoded dataset 

In [0]:
# refs 
# https://learnopencv.com/3d-u-net-brats/#aioseo-dataset-preprocessing (WRT BraTS dataset)
# https://github.com/spmallick/learnopencv/tree/master/Training_3D_U-Net_Brain_Tumor_Seg

# https://nipy.org/nibabel/nifti_images.html

## NIFTI (brain imaging related but not everyone uses it; DICOM may be preferred)
# https://github.com/DataCurationNetwork/data-primers/blob/main/Neuroimaging%20DICOM%20and%20NIfTI%20Data%20Curation%20Primer/neuroimaging-dicom-and-nifti-data-curation-primer.md
# https://discovery.ucl.ac.uk/id/eprint/10146893/1/geometry_medim.pdf 


# test data
# https://www.kaggle.com/datasets/aiocta/brats2023-part-1

In [0]:
# UC path 
# mmt_mlops_demos.cv.data
# /Volumes/mmt_mlops_demos/cv/data/BraTS2021_00495/

In [0]:
## to do -- convert some of the setup as a utils/config file  etc. 

!pip install nibabel -q
!pip install scikit-learn -q
!pip install tqdm -q
!pip install split-folders -q
!pip install torchinfo -q
!pip install segmentation-models-pytorch-3d -q
!pip install livelossplot -q
!pip install torchmetrics -q
!pip install tensorboard -q

!pip install pycocotools
!pip install opencv-python-headless

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import random
import splitfolders
from tqdm import tqdm
import nibabel as nib
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import shutil
import time
 
from dataclasses import dataclass
 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision.transforms as transforms
from torch.cuda import amp
 
from torchmetrics import MeanMetric
from torchmetrics.classification import MulticlassAccuracy
 
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
import gc
 
import segmentation_models_pytorch_3d as smp
 
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot, ExtremaPrinter


In [0]:
# https://www.kaggle.com/datasets/aiocta/brats2023-part-1

# Do ONCE
# !pip install kaggle -q
# !kaggle datasets download -d aiocta/brats2023-part-1 -p /Volumes/mmt_mlops_demos/cv/data/BraTS2023/

In [0]:
# DO ONCE 
# !sudo apt install unzip
# !unzip /Volumes/mmt_mlops_demos/cv/data/BraTS2023/brats2023-part-1.zip -d /Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023-Glioma/

# !rm -rf <UD Vols path to>/brats2023-part-1.zip

In [0]:
# https://www.synapse.org/Synapse:syn51156910/wiki/622351
# Task: Tumor Sub-region Segmentation
# The participants are called to address this task by using the provided clinically-acquired training data to develop their method and produce segmentation labels of the different glioma sub-regions. The sub-regions considered for evaluation are the "enhancing tumor" (ET), the "tumor core" (TC), and the "whole tumor" (WT) [see figure below]. The ET is described by areas that show hyper-intensity in T1Gd when compared to T1, but also when compared to “healthy” white matter in T1Gd. The TC describes the bulk of the tumor, which is what is typically resected. The TC entails the ET, as well as the necrotic (NCR) parts of the tumor. The appearance of NCR is typically hypo-intense in T1-Gd when compared to T1. The WT describes the complete extent of the disease, as it entails the TC and the peritumoral edematous/invaded tissue (ED), which is typically depicted by hyper-intense signal in FLAIR.

# The provided segmentation labels have values of:

# 1 for NCR
# 2 for ED
# 3 for ET
# 0 for everything else.

In [0]:
for s in range(sample_mask_c.shape[0]):
  print(s)
  print(sample_mask_c[s,:,n_slice-ROI_dims[2][0]-1])

In [0]:
output_type= 't2f_128'
output_dir = f'/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_2D/{output_type}/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [0]:
output_dir

In [0]:
import os
import numpy as np
import nibabel as nib
from sklearn.preprocessing import StandardScaler
from PIL import Image

# Define the paths
input_dir = '/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023-Glioma/'

# scan_type = 'tf2'
scan_type = 'seg'

# output_type= 't2f_128'
output_type= f'{scan_type}_128'

output_dir = f'/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_2D/{output_type}/'

# Ensure the output directory exists
# os.makedirs(os.path.join(output_dir, imagesORmasks), exist_ok=True)

# Function to process each NIfTI file
def process_nifti_file(file_path, output_dir, imagesORmasks):
    
    # Ensure the output directory exists
    os.makedirs(os.path.join(output_dir, imagesORmasks), exist_ok=True)

    # Load the NIfTI file
    img = nib.load(file_path)
    img_data = img.get_fdata()

    # Apply scaler fit-transform
    if imagesORmasks == 'images':
        scaler = MinMaxScaler() ## flair ?
        img_data = scaler.fit_transform(img_data.reshape(-1, img_data.shape[-1])).reshape(img_data.shape)

    # Extract ROI dimensions (assuming ROI_dims is a function that returns the dimensions)
    
    ## Define this function based on your requirements
    # ROI_dims = [[56, 184], [56, 184], [13, 141]]
    ROI_range = (slice(56, 184), slice(56, 184), slice(13, 141)) #[56:184, 56:184, 13:141]

    # img_data = img_data[roi_dims]
    img_data = img_data[ROI_range]

    # Convert 3D to 2D slices and save as images
    for i in range(img_data.shape[2]):
        slice_2d = img_data[:, :, i]
        
        # Normalize the slice to 0-255 and convert to uint8
        # slice_2d = ((slice_2d - slice_2d.min()) / (slice_2d.max() - slice_2d.min()) * 255).astype(np.uint8)
        
        # Normalize the slice to 0-255 and convert to uint8
        if slice_2d.max() != slice_2d.min():
            slice_2d = ((slice_2d - slice_2d.min()) / (slice_2d.max() - slice_2d.min()) * 255).astype(np.uint8)
        else:
            slice_2d = np.zeros_like(slice_2d, dtype=np.uint8)
            
        img = Image.fromarray(slice_2d)

        img.save(os.path.join(output_dir, imagesORmasks, f'{os.path.basename(file_path).replace(".nii", "")}_slice_{i}.png'))
            

# # Iterate through all subfolders and process the files
# for root, dirs, files in os.walk(input_dir):
#     for file in files:
#         if file.endswith('t2f.nii'):
#             file_path = os.path.join(root, file)
#             process_nifti_file(file_path, output_dir, imagesORmasks)

In [0]:
import os
import itertools

def nifti_file_iterator(input_dir, scan_type):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            # if file.endswith('t2f.nii'):
            if file.endswith(f'{scan_type}.nii'):
                yield os.path.join(root, file)

# Create an iterator
# scan_type = 't2f' #images
scan_type = 'seg' #masks
file_iterator = nifti_file_iterator(input_dir,scan_type)

# # Process each file using the iterator
# for file_path in file_iterator:
#     print(file_path)
#     process_nifti_file(file_path, output_dir, imagesORmasks)

In [0]:
file_iterator.__next__()

In [0]:
file_iterator.__next__()

In [0]:
# input_fpath = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023-Glioma/BraTS-GLI-00000-000/BraTS-GLI-00000-000-t2f.nii"

# imagesORmasks = 'images'

# process_nifti_file(input_fpath, output_dir, imagesORmasks)

In [0]:
input_fpath = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023-Glioma/BraTS-GLI-00000-000/BraTS-GLI-00000-000-seg.nii"

imagesORmasks = 'masks'

process_nifti_file(input_fpath, output_dir, imagesORmasks)

In [0]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import re

# Define the path to the directory containing the sliced images
sliced_images_dir = '/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_2D/t2f_128/images'

# Function to extract numerical part from the filename for sorting
def extract_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else 0

# Get a list of all image files in the directory
# image_files = [f for f in os.listdir(sliced_images_dir) if f.endswith('.png')]

# Sort the image files to ensure they are in the correct numerical order
# image_files.sort(key=extract_number)
image_files = [f'BraTS-GLI-00000-000-t2f_slice_{n}.png' for n in range(128)]

# Function to display a grid of images
def display_images(image_files, num_images=128):
    plt.figure(figsize=(12, 50))
    for i, image_file in enumerate(image_files[:num_images]):
        img = Image.open(os.path.join(sliced_images_dir, image_file))
        plt.subplot(23, 6, i + 1)
        plt.imshow(img, cmap='gray')
        plt.title(image_file)
        plt.axis('off')
    plt.show()

# Display the first X images
display_images(image_files)

In [0]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import re

# Define the path to the directory containing the sliced images
sliced_images_dir = '/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_2D/seg_128/masks'

# Function to extract numerical part from the filename for sorting
def extract_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else 0

# Get a list of all image files in the directory
# image_files = [f for f in os.listdir(sliced_images_dir) if f.endswith('.png')]

# Sort the image files to ensure they are in the correct numerical order
# image_files.sort(key=extract_number)
image_files = [f'BraTS-GLI-00000-000-seg_slice_{n}.png' for n in range(128)]

# Function to display a grid of images
def display_images(image_files, num_images=128):
    plt.figure(figsize=(12, 50))
    for i, image_file in enumerate(image_files[:num_images]):
        img = Image.open(os.path.join(sliced_images_dir, image_file))
        plt.subplot(23, 6, i + 1)
        plt.imshow(img, cmap='gray')
        plt.title(image_file)
        plt.axis('off')
    plt.show()

# Display the first X images
display_images(image_files)

In [0]:
import numpy as np
from PIL import Image

# Define the path to the specific image
image_path = os.path.join(sliced_images_dir, 'BraTS-GLI-00000-000-seg_slice_60.png')

# Open the image using PIL
img = Image.open(image_path)

# Convert the image to a NumPy array
img_array = np.array(img)

# Display the shape of the NumPy array
print(img_array.shape)

In [0]:
plt.imshow(img_array)

In [0]:
for i in range(128):
  print(img_array[i,:])

In [0]:
import numpy as np

((np.array([0, 85, 170, 255])-0)/255)*3 

In [0]:
# # re.sub(r'\d+', '{n}', image_files[0])

# import os
# import re

# # Define the path to the directory containing the sliced images
# sliced_images_dir = '/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_2D/t2f_128/images'

# # Function to replace numeric strings with different letters
# def replace_numbers_with_letters(filename):
#     return re.sub(r'\d+', lambda x: chr(97 + int(x.group())), filename)

# # Get a list of all image files in the directory
# image_files = [f for f in os.listdir(sliced_images_dir) if f.endswith('.png')]

# # Extract the main file name pattern and replace numeric strings with letters
# pattern_files = [replace_numbers_with_letters(f) for f in image_files]

# # Display the modified file names
# pattern_files

In [0]:
[f'BraTS-GLI-00000-000-t2f_slice_{n}.png' for n in range(128)]

In [0]:
# to continue

In [0]:
%python
import json
import os
from PIL import Image
import numpy as np
import cv2

# Define the categories
categories = [
    {"id": 1, "name": "NCR"},   
    {"id": 2, "name": "ED"},
    {"id": 3, "name": "ET"},
    {"id": 0, "name": "nonT"}
]

# Initialize the COCO dataset structure
coco_dataset = {
    "images": [],
    "annotations": [],
    "categories": categories
}

images_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/images/"
# masks_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/masks/"

# Assuming images and masks are numpy arrays
image_id = 0
annotation_id = 0

for i in range(len(images)):
    image = images[i].numpy()  # Convert Tensor to NumPy array
    mask = masks[i].numpy()  # Convert Tensor to NumPy array

    # Convert image to uint8 and reshape
    image = image.astype(np.uint8).squeeze()

    # Ensure the image is 2D or 3D
    if image.ndim == 4:
        image = image[0, 0]  # Adjust this based on your specific data shape

    # Save the image
    image_filename = f"image_{image_id}.npy"
    image_path = os.path.join(images_path, image_filename)
    Image.fromarray(image).save(image_path)

    # Add image info to COCO dataset
    coco_dataset["images"].append({
        "id": image_id,
        "file_name": image_filename,
        "height": image.shape[0],
        "width": image.shape[1]
    })

    # Process the mask
    for category_id in range(1, 5):
        category_mask = (mask == category_id).astype(np.uint8)

        # Find contours (assuming OpenCV is available)
        contours, _ = cv2.findContours(category_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            segmentation = contour.flatten().tolist()
            x, y, w, h = cv2.boundingRect(contour)
            area = cv2.contourArea(contour)

            # Add annotation info to COCO dataset
            coco_dataset["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "segmentation": [segmentation],
                "area": area,
                "bbox": [x, y, w, h],
                "iscrowd": 0
            })
            annotation_id += 1

    image_id += 1

# Save the COCO dataset to a JSON file
with open("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed/input_data_128/val/images/instances_train2017.json", "w") as f:
    json.dump(coco_dataset, f)

In [0]:
# t1ce_list = sorted(glob.glob(f"{DATASET_PATH}/*/*t1c.nii"))
# t2_list = sorted(glob.glob(f"{DATASET_PATH}/*/*t2w.nii"))
# flair_list = sorted(glob.glob(f"{DATASET_PATH}/*/*t2f.nii"))
# mask_list = sorted(glob.glob(f"{DATASET_PATH}/*/*seg.nii"))


# print("t1ce list: ", len(t1ce_list))
# print("t2 list: ", len(t2_list))
# print("flair list: ", len(flair_list))
# print("Mask list: ", len(mask_list))

# # t1ce list:  625
# # t2 list:  625
# # flair list:  625
# # Mask list:  625

In [0]:
# import json

# # Save the lists to UC volume
# dbutils.fs.put("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/t1ce_list.json", json.dumps(t1ce_list), overwrite=True)
# dbutils.fs.put("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/t2_list.json", json.dumps(t2_list), overwrite=True)
# dbutils.fs.put("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/flair_list.json", json.dumps(flair_list), overwrite=True)
# dbutils.fs.put("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/mask_list.json", json.dumps(mask_list), overwrite=True)

In [0]:
import json

# Read the JSON files from the UC volume using Unix commands
t1ce_list_json = dbutils.fs.head("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/t1ce_list.json", 1000000)
t2_list_json = dbutils.fs.head("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/t2_list.json", 1000000)
flair_list_json = dbutils.fs.head("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/flair_list.json", 1000000)
mask_list_json = dbutils.fs.head("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/mask_list.json", 1000000)

# Parse the JSON strings into Python lists
t1ce_list = json.loads(t1ce_list_json)
t2_list = json.loads(t2_list_json)
flair_list = json.loads(flair_list_json)
mask_list = json.loads(mask_list_json)

# Print the lengths of the lists
print("t1ce list: ", len(t1ce_list))
print("t2 list: ", len(t2_list))
print("flair list: ", len(flair_list))
print("Mask list: ", len(mask_list))

In [0]:
# mask_list

In [0]:
# to continue with preprocessing for normal pytorch process and then try to convert to coco 

In [0]:
## DATASET Preprocessing test to pytorch dataloader 
# -- we will need to see how to reformat to coco/yolo friendly format 

In [0]:
# '/'.join(f"{DATASET_PATH}".split("/")[:-2])
UCV_folderpath =  "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/"
# UCV_subfoldername = "BraTS2023_Preprocessed"
# UCV_subfoldername = "BraTS2023_Preprocessed_v2" ## without one-hot-mask encoding

UCV_subfoldername = "BraTS2023_Preprocessed_2D" ## without one-hot-mask encoding

In [0]:
## Do Once

for idx in tqdm(
    range(len(t2_list)), desc="Preparing to stack, crop and save", unit="file"
):
    temp_image_t1ce = nib.load(t1ce_list[idx]).get_fdata()
    temp_image_t1ce = scaler.fit_transform(
        temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])
    ).reshape(temp_image_t1ce.shape)
 
    temp_image_t2 = nib.load(t2_list[idx]).get_fdata()
    temp_image_t2 = scaler.fit_transform(
        temp_image_t2.reshape(-1, temp_image_t2.shape[-1])
    ).reshape(temp_image_t2.shape)
 
    temp_image_flair = nib.load(flair_list[idx]).get_fdata()
    temp_image_flair = scaler.fit_transform(
        temp_image_flair.reshape(-1, temp_image_flair.shape[-1])
    ).reshape(temp_image_flair.shape)
 
    temp_combined_images = np.stack(
        [temp_image_flair, temp_image_t1ce, temp_image_t2], axis=3
    )
    temp_combined_images = temp_combined_images[56:184, 56:184, 13:141]

    temp_mask = nib.load(mask_list[idx]).get_fdata()
    temp_mask = temp_mask[56:184, 56:184, 13:141]
 
    val, counts = np.unique(temp_mask, return_counts=True)
 
    # If a volume has less than 1% of mask, we simply ignore to reduce computation
    if (1 - (counts[0] / counts.sum())) > 0.01:
        #         print("Saving Processed Images and Masks")
        
        ## omit the one_hot encoding ?
        if UCV_subfoldername != "BraTS2023_Preprocessed_v2":
            temp_mask = F.one_hot(torch.tensor(temp_mask, dtype=torch.long), num_classes=4)
        
        os.makedirs(f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/images", exist_ok=True)
        os.makedirs(f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/masks", 
                    exist_ok=True)
 
        np.save(
            f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/images/image_"
            + str(idx)
            + ".npy",
            temp_combined_images,
        )
        np.save(
            f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/masks/mask_"
            + str(idx)
            + ".npy",
            temp_mask,
        )
 
    else:
        pass

In [0]:
###

In [0]:
# compare tqdm vs vectorized pandasUDF for processing

In [0]:
# from pyspark.sql.functions import pandas_udf, PandasUDFType
# import pandas as pd
# import numpy as np
# import nibabel as nib
# import os
# import torch
# import torch.nn.functional as F
# from typing import List

# # Define the UDF
# @pandas_udf("string", PandasUDFType.SCALAR)
# def process_images(t1ce_path: pd.Series, 
#                    t2_path: pd.Series, 
#                    flair_path: pd.Series, 
#                    mask_path: pd.Series, 
#                    idx: pd.Series
#                    ) -> pd.Series:
#     results: List[str] = []
#     for i in range(len(t1ce_path)):
#         temp_image_t1ce: np.ndarray = nib.load(t1ce_path[i]).get_fdata()
#         temp_image_t1ce = scaler.fit_transform(
#             temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])
#         ).reshape(temp_image_t1ce.shape)

#         temp_image_t2: np.ndarray = nib.load(t2_path[i]).get_fdata()
#         temp_image_t2 = scaler.fit_transform(
#             temp_image_t2.reshape(-1, temp_image_t2.shape[-1])
#         ).reshape(temp_image_t2.shape)

#         temp_image_flair: np.ndarray = nib.load(flair_path[i]).get_fdata()
#         temp_image_flair = scaler.fit_transform(
#             temp_image_flair.reshape(-1, temp_image_flair.shape[-1])
#         ).reshape(temp_image_flair.shape)

#         temp_mask: np.ndarray = nib.load(mask_path[i]).get_fdata()

#         temp_combined_images: np.ndarray = np.stack(
#             [temp_image_flair, temp_image_t1ce, temp_image_t2], axis=3
#         )

#         temp_combined_images = temp_combined_images[56:184, 56:184, 13:141]
#         temp_mask = temp_mask[56:184, 56:184, 13:141]

#         val, counts = np.unique(temp_mask, return_counts=True)

#         # If a volume has less than 1% of mask, we simply ignore to reduce computation
#         if (1 - (counts[0] / counts.sum())) > 0.01:
#             if UCV_subfoldername != "BraTS2023_Preprocessed_v2":
#                 temp_mask = F.one_hot(torch.tensor(temp_mask, dtype=torch.long), num_classes=4).numpy()
            
#             images_dir: str = f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/images"
#             masks_dir: str = f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/masks"
#             os.makedirs(images_dir, exist_ok=True)
#             os.makedirs(masks_dir, exist_ok=True)

#             np.save(
#                 f"{images_dir}/image_" + str(idx[i]) + ".npy",
#                 temp_combined_images,
#             )
#             np.save(
#                 f"{masks_dir}/mask_" + str(idx[i]) + ".npy",
#                 temp_mask,
#             )
#             results.append("Processed")
#         else:
#             results.append("Skipped")
#     return pd.Series(results)

# # Create a Spark DataFrame
# data = list(zip(t1ce_list, t2_list, flair_list, mask_list, range(len(t1ce_list))))
# columns = ["t1ce_path", "t2_path", "flair_path", "mask_path", "idx"]
# df = spark.createDataFrame(data, columns)

# # Apply the UDF
# df = df.withColumn("result", process_images("t1ce_path", "t2_path", "flair_path", "mask_path", "idx"))

# display(df)

In [0]:
images_folder = f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/images"
print(len(os.listdir(images_folder)))
 
masks_folder = f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/masks"
print(len(os.listdir(masks_folder)))

# Images: 575
# Masks: 575

In [0]:
masks_folder

In [0]:
os.listdir(masks_folder)

In [0]:
os.listdir(images_folder)

In [0]:
os.listdir(images_path)[0]

In [0]:
os.path.join(os.listdir(images_path)[0], 'na')

In [0]:
%python
import numpy as np
from PIL import Image
import os

images_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/images/"
jpegs_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/jpeg/"

# Create directories if they do not exist
os.makedirs(images_path, exist_ok=True)
os.makedirs(jpegs_path, exist_ok=True)

# Iterate over all .npy files in the images_path directory
for npy_file in os.listdir(images_path):
    if npy_file.endswith(".npy"):
        npy_file_path = os.path.join(images_path, npy_file)
        
        # Load the .npy file
        image_array = np.load(npy_file_path)

        # Ensure the image array is in the correct format (e.g., uint8)
        image_array = image_array.astype(np.uint8)

        # Check if the array is 3D
        if image_array.ndim == 3:
            # Convert the numpy array to a PIL Image
            image = Image.fromarray(image_array)
            
            # Save the image as a JPEG file
            jpeg_file_name = npy_file.replace(".npy", ".jpeg")
            jpeg_file_path = os.path.join(jpegs_path, jpeg_file_name)
            image.save(jpeg_file_path)
        else:
            print(f"Skipping file {npy_file} due to incompatible shape: {image_array.shape}")

In [0]:

input_folder = f"{UCV_folderpath}{UCV_subfoldername}/input_data_3channels/"
 
output_folder = f"{UCV_folderpath}{UCV_subfoldername}/input_data_128/"

## do once [?]
splitfolders.ratio(
    input_folder, output_folder, seed=42, ratio=(0.75, 0.25), group_prefix=None
    # input_folder, output_folder, seed=42, ratio=(0.7, 0.2, 0.1), group_prefix=None
)

In [0]:
class BraTSDataset(Dataset):
    def __init__(self, img_dir, mask_dir, normalization=True):
        super().__init__()
 
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_list = sorted(
            os.listdir(img_dir)
        )  # Ensure sorting to match images and masks
        self.mask_list = sorted(os.listdir(mask_dir))
        self.normalization = normalization
 
        # If normalization is True, set up a normalization transform
        if self.normalization:
            self.normalizer = transforms.Normalize(
                mean=[0.5], std=[0.5]
            )  # Adjust mean and std based on your data
 
    def load_file(self, filepath):
        return np.load(filepath)
 
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
       image_path = os.path.join(self.img_dir, self.img_list[idx])
       mask_path = os.path.join(self.mask_dir, self.mask_list[idx])
       # Load the image and mask
       image = self.load_file(image_path)
       mask = self.load_file(mask_path)
 
       # Convert to torch tensors and permute axes to C, D, H, W format (needed for 3D models)
       image = torch.from_numpy(image).permute(3, 2, 0, 1)  # Shape: C, D, H, W

    #    mask = torch.from_numpy(mask).permute(3, 2, 0, 1)  # Shape: C, D, H, W

       mask = torch.from_numpy(mask).permute(2, 0, 1).unsqueeze(0)  # Shape: 1, D, H, W (added channel dimension)   

    #    mask = torch.from_numpy(mask).permute(2, 0, 1)  # Shape: D, H, W 

    #    if masks.dim() == 4:
       
    #    elif masks.dim() == 3:
            # mask = torch.from_numpy(mask).permute(2, 0, 1)  # Shape: D, H, W

    #    mask = torch.from_numpy(mask).permute(2, 0, 1).unsqueeze(0)  # Shape: 1, D, H, W (added channel dimension)
       
       # Normalize the image if normalization is enabled
       if self.normalization:
           image = self.normalizer(image)
       
       return image, mask

In [0]:
# class BraTSDataset(Dataset):
#     def __init__(self, img_dir, mask_dir, normalization=True):
#         super().__init__()
 
#         self.img_dir = img_dir
#         self.mask_dir = mask_dir
#         self.img_list = sorted(
#             os.listdir(img_dir)
#         )  # Ensure sorting to match images and masks
#         self.mask_list = sorted(os.listdir(mask_dir))
#         self.normalization = normalization
 
#         # If normalization is True, set up a normalization transform
#         if self.normalization:
#             self.normalizer = transforms.Normalize(
#                 mean=[0.5], std=[0.5]
#             )  # Adjust mean and std based on your data
 
#     def load_file(self, filepath):
#         return np.load(filepath)
 
#     def __len__(self):
#         return len(self.img_list)
    
#     def __getitem__(self, idx):
#        image_path = os.path.join(self.img_dir, self.img_list[idx])
#        mask_path = os.path.join(self.mask_dir, self.mask_list[idx])
#        # Load the image and mask
#        image = self.load_file(image_path)
#        mask = self.load_file(mask_path)
 
#        # Convert to torch tensors and permute axes to C, D, H, W format (needed for 3D models)
#        image = torch.from_numpy(image).permute(3, 2, 0, 1)  # Shape: C, D, H, W
#        mask = torch.from_numpy(mask).permute(3, 2, 0, 1)  # Shape: C, D, H, W
       
#        # Normalize the image if normalization is enabled
#        if self.normalization:
#            image = self.normalizer(image)
       
#        return image, mask

In [0]:
train_img_dir = f"{UCV_folderpath}{UCV_subfoldername}/input_data_128/train/images"
train_mask_dir = f"{UCV_folderpath}{UCV_subfoldername}/input_data_128/train/masks"
 
val_img_dir = f"{UCV_folderpath}{UCV_subfoldername}/input_data_128/val/images"
val_mask_dir = f"{UCV_folderpath}{UCV_subfoldername}/input_data_128/val/masks"
 
val_img_list = os.listdir(val_img_dir)
val_mask_list = os.listdir(val_mask_dir)
 
# Initialize datasets with normalization only
train_dataset = BraTSDataset(train_img_dir, train_mask_dir, normalization=True)
val_dataset = BraTSDataset(val_img_dir, val_mask_dir, normalization=True)
 
# Print dataset statistics
print("Total Training Samples: ", len(train_dataset))
print("Total Val Samples: ", len(val_dataset))

#Total Training Samples:  431
#Total Val Samples:  144

In [0]:
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, num_workers=4)

# Sanity Check
images, masks = next(iter(train_loader))
print(f"Train Image batch shape: {images.shape}")
print(f"Train Mask batch shape: {masks.shape}")

# Train Image batch shape: torch.Size([5, 3, 128, 128, 128])
# Train Mask batch shape: torch.Size([5, 4, 128, 128, 128])


# Train Image batch shape: torch.Size([5, 3, 128, 128, 128])
# Train Mask batch shape: torch.Size([5, 1, 128, 128, 128])

# Train Image batch shape: torch.Size([5, 3, 128, 128, 128])
# Train Mask batch shape: torch.Size([5, 128, 128, 128])

In [0]:
middle_slice = images.shape[2] // 2
print(middle_slice)
plt.imshow(masks[0, 0, middle_slice, :, :], cmap="viridis")

In [0]:
# masks.shape == torch.Size([5, 4, 128, 128, 128])
masks.shape == torch.Size([5, 1, 128, 128, 128])

In [0]:
def visualize_slices(images, masks, num_slices=20):
    batch_size = images.shape[0]
 
    if masks.shape == torch.Size([5, 4, 128, 128, 128]):
        masks = torch.argmax(masks, dim=1)  # along the channel/class dim -- predicted value of each class for that dim (class)
        
    ## no longer needed when all is already "combined" in the mask
 
    for i in range(min(num_slices, batch_size)):
        fig, ax = plt.subplots(1, 5, figsize=(15, 5))
 
        middle_slice = images.shape[2] // 2
        
        ax[0].imshow(images[i, 0, middle_slice, :, :], cmap="gray")
        ax[1].imshow(images[i, 1, middle_slice, :, :], cmap="gray")
        ax[2].imshow(images[i, 2, middle_slice, :, :], cmap="gray")
        
        # ax[3].imshow(masks[i, middle_slice, :, :], cmap="viridis")
        # ax[4].imshow(masks[i, middle_slice, :, :], cmap="gray")
        ax[3].imshow(masks[i,0, middle_slice, :, :], cmap="viridis")
        ax[4].imshow(masks[i,0, middle_slice, :, :], cmap="gray")

        ax[0].set_title("T1ce")
        ax[1].set_title("FLAIR")
        ax[2].set_title("T2")
        ax[3].set_title("Seg Mask")
        ax[4].set_title("Mask - Gray")
 
        plt.show()
 
 
visualize_slices(images, masks, num_slices=20)

In [0]:
images, masks = next(iter(train_loader))
visualize_slices(images, masks, num_slices=20)

In [0]:
# def visualize_slices(images, masks, num_slices=20):
#     batch_size = images.shape[0]
 
#     ## if torch.Size([5, 4, 128, 128, 128])
#     masks = torch.argmax(masks, dim=1)  # along the channel/class dim
 
#     for i in range(min(num_slices, batch_size)):
#         fig, ax = plt.subplots(1, 5, figsize=(15, 5))
 
#         middle_slice = images.shape[2] // 2
#         ax[0].imshow(images[i, 0, middle_slice, :, :], cmap="gray")
#         ax[1].imshow(images[i, 1, middle_slice, :, :], cmap="gray")
#         ax[2].imshow(images[i, 2, middle_slice, :, :], cmap="gray")
                
#         ax[3].imshow(masks[i, middle_slice, :, :], cmap="viridis")
#         ax[4].imshow(masks[i, middle_slice, :, :], cmap="gray")
#         # ax[3].imshow(masks[i][middle_slice, :, :], cmap="viridis")
#         # ax[4].imshow(masks[i][middle_slice, :, :], cmap="gray")

        
#         ax[0].set_title("T1ce")
#         ax[1].set_title("FLAIR")
#         ax[2].set_title("T2")
#         ax[3].set_title("Seg Mask")
#         ax[4].set_title("Mask - Gray")
 
#         plt.show()
 
 
# visualize_slices(images, masks, num_slices=20)

In [0]:
masks[3][:,middle_slice,:,:]

In [0]:
import matplotlib.pyplot as plt

# Assuming middle_slice is already defined
middle_slice = images.shape[2] // 2

# Convert the mask to a 2D array
mask_2d = masks[0, 0, middle_slice, :, :]
# mask_2d = masks[0][0,middle_slice,:,:]

# Plot the 2D mask
plt.imshow(mask_2d, cmap="viridis")
plt.title(f"Mask for middle slice {middle_slice}")
plt.colorbar()
plt.show()

In [0]:
# images, masks = next(iter(train_loader))
# visualize_slices(images, masks, num_slices=20)

In [0]:
# ## Original Viz code
# def visualize_slices(images, masks, num_slices=20):
#     batch_size = images.shape[0]
 
#     masks = torch.argmax(masks, dim=1)  # along the channel/class dim
 
#     for i in range(min(num_slices, batch_size)):
#         fig, ax = plt.subplots(1, 5, figsize=(15, 5))
 
#         middle_slice = images.shape[2] // 2
#         ax[0].imshow(images[i, 0, middle_slice, :, :], cmap="gray")
#         ax[1].imshow(images[i, 1, middle_slice, :, :], cmap="gray")
#         ax[2].imshow(images[i, 2, middle_slice, :, :], cmap="gray")

#         ax[3].imshow(masks[i, middle_slice, :, :], cmap="viridis")
#         ax[4].imshow(masks[i, middle_slice, :, :], cmap="gray")
 
#         ax[0].set_title("T1ce")
#         ax[1].set_title("FLAIR")
#         ax[2].set_title("T2")
#         ax[3].set_title("Seg Mask")
#         ax[4].set_title("Mask - Gray")
 
#         plt.show()
 
 
# visualize_slices(images, masks, num_slices=20)

In [0]:
!pip install pycocotools
!pip install opencv-python-headless


In [0]:
images.shape, masks.shape

In [0]:
masks[0]

In [0]:
# # 1 for NCR
# # 2 for ED
# # 3 for ET
# # 0 for everything else.

# categories = ["nonT", "NCR", "ED", "ET"]

In [0]:
images_folder

In [0]:
%python
import json
import os
from PIL import Image
import numpy as np
import cv2

# Define the categories
categories = [
    {"id": 1, "name": "NCR"},   
    {"id": 2, "name": "ED"},
    {"id": 3, "name": "ET"},
    {"id": 0, "name": "nonT"}
]

# Initialize the COCO dataset structure
coco_dataset = {
    "images": [],
    "annotations": [],
    "categories": categories
}

images_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/images/"
# masks_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/masks/"

# Assuming images and masks are numpy arrays
image_id = 0
annotation_id = 0

for i in range(len(images)):
    image = images[i].numpy()  # Convert Tensor to NumPy array
    mask = masks[i].numpy()  # Convert Tensor to NumPy array

    # Convert image to uint8 and reshape
    image = image.astype(np.uint8).squeeze()

    # Ensure the image is 2D or 3D
    if image.ndim == 4:
        image = image[0, 0]  # Adjust this based on your specific data shape

    # Save the image
    image_filename = f"image_{image_id}.npy"
    image_path = os.path.join(images_path, image_filename)
    Image.fromarray(image).save(image_path)

    # Add image info to COCO dataset
    coco_dataset["images"].append({
        "id": image_id,
        "file_name": image_filename,
        "height": image.shape[0],
        "width": image.shape[1]
    })

    # Process the mask
    for category_id in range(1, 5):
        category_mask = (mask == category_id).astype(np.uint8)

        # Find contours (assuming OpenCV is available)
        contours, _ = cv2.findContours(category_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            segmentation = contour.flatten().tolist()
            x, y, w, h = cv2.boundingRect(contour)
            area = cv2.contourArea(contour)

            # Add annotation info to COCO dataset
            coco_dataset["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "segmentation": [segmentation],
                "area": area,
                "bbox": [x, y, w, h],
                "iscrowd": 0
            })
            annotation_id += 1

    image_id += 1

# Save the COCO dataset to a JSON file
with open("/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed/input_data_128/val/images/instances_train2017.json", "w") as f:
    json.dump(coco_dataset, f)

In [0]:
%python
import json
import os
import numpy as np
from PIL import Image
import cv2

# Define the categories
categories = [
    {"id": 1, "name": "NCR"},
    {"id": 2, "name": "ED"},
    {"id": 3, "name": "ET"},
    {"id": 0, "name": "nonT"}
]

# Initialize the COCO dataset structure
coco_dataset = {
    "images": [],
    "annotations": [],
    "categories": categories
}


images_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/images/"
# masks_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/val/masks/"
output_json_path = "/Volumes/mmt_mlops_demos/cv/data/BraTS2023/BraTS2023_Preprocessed_v2/input_data_128/test/instances.json"

# Create directories if they do not exist
os.makedirs(images_path, exist_ok=True)
os.makedirs(masks_path, exist_ok=True)

image_id = 0
annotation_id = 0

# Iterate over all .npy files in the images_path directory
for npy_file in os.listdir(images_path):
    if npy_file.endswith(".npy"):
        npy_file_path = os.path.join(images_path, npy_file)
        
        # Load the .npy file
        image_array = np.load(npy_file_path)

        # Ensure the image array is in the correct format (e.g., uint8)
        image_array = image_array.astype(np.uint8)

        # Check if the array is 3D
        if image_array.ndim == 3:
            # Convert the numpy array to a PIL Image
            image = Image.fromarray(image_array)
            
            # Save the image as a JPEG file
            jpeg_file_name = npy_file.replace(".npy", ".jpeg")
            jpeg_file_path = os.path.join(images_path, jpeg_file_name)
            image.save(jpeg_file_path)

            # Add image info to COCO dataset
            coco_dataset["images"].append({
                "id": image_id,
                "file_name": jpeg_file_name,
                "height": image_array.shape[0],
                "width": image_array.shape[1]
            })

            # Process the mask
            mask_file_path = os.path.join(masks_path, npy_file)
            mask_array = np.load(mask_file_path).astype(np.uint8)

            for category_id in range(1, 5):
                category_mask = (mask_array == category_id).astype(np.uint8)

                # Find contours (assuming OpenCV is available)
                contours, _ = cv2.findContours(category_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for contour in contours:
                    segmentation = contour.flatten().tolist()
                    x, y, w, h = cv2.boundingRect(contour)
                    area = cv2.contourArea(contour)

                    # Add annotation info to COCO dataset
                    coco_dataset["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": category_id,
                        "segmentation": [segmentation],
                        "area": area,
                        "bbox": [x, y, w, h],
                        "iscrowd": 0
                    })
                    annotation_id += 1

            image_id += 1
        else:
            print(f"Skipping file {npy_file} due to incompatible shape: {image_array.shape}")

# # Save the COCO dataset to a JSON file
# with open(output_json_path, "w") as f:
#     json.dump(coco_dataset, f)